<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/RNA_Workshop/Analyze_BMX_scRNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install scanpy --quiet
import h5py
import numpy as np
import scipy as sci
from matplotlib import pyplot as plt
import scanpy as sc
import tarfile
import os
import anndata as ad
import pandas as pd
import pybiomart
import shutil
from tqdm import tqdm
import urllib.request
from IPython.display import clear_output
from matplotlib.pyplot import rc_context
from scipy import stats as st
import logging
logger = logging.basicConfig(level=logging.INFO,format='%(asctime)s  %(message)s', datefmt='%Y/%m/%d %I:%M:%S %p',force=True)



In [15]:
def z_score(x,axis=-1):
    x=np.array(x)
    return (x-np.mean(x,axis=axis))/np.std(x,axis=axis)

def publishable_plots(FS=6):
    plt.rcParams.update({'font.size': FS,'axes.linewidth':.5,'figure.dpi':300,
                         'xtick.major.width': 0.5,'ytick.major.width': 0.5,
                         'figure.titlesize':FS,'axes.titlesize': FS,'xtick.labelsize': FS,
                         'ytick.labelsize':FS,'axes.labelsize': FS,'legend.fontsize': FS,
                         'figure.labelsize':FS})

    import urllib.request
    arial_link = 'https://raw.githubusercontent.com/dtabuena/Resources/main/Fonts/arial.ttf'
    filename = './arial.ttf'
    urllib.request.urlretrieve(arial_link, filename)
    plt.rcParams.update({'font.family': 'arial'})

    try:
        sc.settings.verbosity = 'error'             # verbosity: errors (0), warnings (1), info (2), hints (3)
        sc.settings.set_figure_params(dpi=300, facecolor='white',fontsize=FS)
    except: None
    return None


def pull_gene_annots(csv_loc='./mmusculus_coding_noncoding.csv',
                     my_git='https://raw.githubusercontent.com/dtabuena/Resources/main/Genetics/mmusculus_coding_noncoding.csv',
                     biomart_name='mmusculus',
                     biomart_keys=["ensembl_gene_id", "chromosome_name","transcript_biotype","external_gene_name","peptide"]):

    logging.info('pulling gene annotations...')
    if os.path.exists('./mmusculus_coding_noncoding.csv'):
        logging.info( 'Use local copy of musmus')
        annot_dd = pd.read_csv('./mmusculus_coding_noncoding.csv').set_index("external_gene_name")
    else:
        try:
            logging.info( 'attempting to pull mus mus from git...')
            musmus_link = 'https://raw.githubusercontent.com/dtabuena/Resources/main/Genetics/mmusculus_coding_noncoding.csv'
            filename = './mmusculus_coding_noncoding.csv'
            urllib.request.urlretrieve(musmus_link, filename)
            annot_dd = pd.read_csv(filename).set_index("external_gene_name")
        except:
            logging.info('attempting to pull mus mus from biomart...')
            annot = sc.queries.biomart_annotations("mmusculus",["ensembl_gene_id", "chromosome_name","transcript_biotype","external_gene_name"],).set_index('ensembl_gene_id')
            uniq_inds = list(set(list(annot.index)))
            for r in tqdm(uniq_inds):
                match_bool = annot.index.str.contains(r)
                if np.sum(match_bool)>1:
                    new_val ='__'.join(list(annot.loc[r,'transcript_biotype']))
                    annot.at[r,'transcript_biotype']=new_val
            annot['is_coding']= annot.transcript_biotype.str.contains('coding')
            annot_dd = annot.drop_duplicates().set_index("external_gene_name")
            annot_dd.to_csv('./mmusculus_coding_noncoding.csv')

    coding_list = annot_dd.index[ annot_dd['is_coding'] ].to_list()
    return coding_list, annot_dd


def copy_original_data():
    ##### Find and Copy MTX DAta
    subfolder='/outs/raw_gene_bc_matrices/mm10-1.2.0_premrna/'
    parent_folder='//hive.gladstone.internal/Huang-Lab/kzalocus/bumex_snSeq/'
    needed_files = ['/barcodes.tsv', '/genes.tsv', '/matrix.mtx']
    dest_folder = 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/snBMX/'

    folder_list = [f.name for f in os.scandir(parent_folder) if f.is_dir()]
    folder_list = [f for f in folder_list if "_" in f]
    _=[logging.info(f) for f in folder_list if "_" in f]


    f = folder_list[0]
    f_list = [''.join([parent_folder,f,subfolder,n]) for n in needed_files] #
    logging.info(f_list)

    for f in folder_list:
        for n in needed_files:
            src = ''.join([parent_folder,f,subfolder,n])
            try:
                logging.info(''.join([dest_folder,f,'/']))
                os.makedirs(''.join([dest_folder,f,'/']))
            except: None
            dst = ''.join([dest_folder,f,n])
            logging.info(src)
            logging.info(dst)
            shutil.copyfile(src, dst)
    return None


publishable_plots(6)
pd.set_option('display.max_rows',100)

In [16]:
copy_original_data()


2023/10/10 11:32:45 AM  01_E3Ki3611
2023/10/10 11:32:45 AM  02_E3Ki3607
2023/10/10 11:32:45 AM  03_E3Ki3608
2023/10/10 11:32:45 AM  04_E3Ki3612
2023/10/10 11:32:45 AM  05_E3Ki3613
2023/10/10 11:32:45 AM  06_E3Ki3614
2023/10/10 11:32:45 AM  07_E4Ki2737
2023/10/10 11:32:45 AM  08_E4Ki2738
2023/10/10 11:32:45 AM  09_E4Ki2739
2023/10/10 11:32:45 AM  10_E4Ki2710
2023/10/10 11:32:45 AM  11_E4Ki2711
2023/10/10 11:32:45 AM  12_E4Ki2712
2023/10/10 11:32:45 AM  ['//hive.gladstone.internal/Huang-Lab/kzalocus/bumex_snSeq/01_E3Ki3611/outs/raw_gene_bc_matrices/mm10-1.2.0_premrna//barcodes.tsv', '//hive.gladstone.internal/Huang-Lab/kzalocus/bumex_snSeq/01_E3Ki3611/outs/raw_gene_bc_matrices/mm10-1.2.0_premrna//genes.tsv', '//hive.gladstone.internal/Huang-Lab/kzalocus/bumex_snSeq/01_E3Ki3611/outs/raw_gene_bc_matrices/mm10-1.2.0_premrna//matrix.mtx']
2023/10/10 11:32:45 AM  C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/snBMX/01_E3Ki3611/
2023/10/10 11:32:45 AM  //hive.gladstone.internal/Huang-L

In [17]:
key_loc = 'https://raw.githubusercontent.com/dtabuena/Workshop/61a957d8c34d5c46c6ab7cc52e1f0330607aaf86/RNA_Workshop/UNOFFICIAL_apoE_BMX_key.csv'
data_key = pd.read_csv(key_loc).set_index('Animal')
display(data_key)

,Treatment
Animal,
E3Ki3607,BMX
E3Ki3608,BMX
E3Ki3611,BMX
E3Ki3612,VEH
E3Ki3613,VEH
E3Ki3614,VEH
E4Ki2710,VEH
E4Ki2711,VEH
E4Ki2712,VEH


In [19]:
working_dir = 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/snBMX/'
os.chdir(working_dir)
folder_list = [f.path for f in os.scandir(working_dir) if f.is_dir() and '_' in f.name]
if len(folder_list)<2:
    copy_original_data()
    folder_list = [f.path for f in os.scandir(working_dir) if f.is_dir() and '_' in f.name]
folder_list = [f for f in folder_list if 'DEG' not in f]
adata_dict = {}
for f in tqdm(folder_list):
    a = sc.read_10x_mtx(f)
    sample_ID = f.split('/')[-1]
    adata_dict[sample_ID]=a
    a=[]
adata = ad.concat(adata_dict,axis = 0,label="sample",index_unique="_")




100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:08<00:00,  5.68s/it]


In [ ]:
adata.obs['treatment'] = [ data_key.loc[bc.split('_')[-1],'Treatment']  for bc in adata.obs.index]
adata.obs['apoe_iso'] = [s.split('_')[-1][:4] for s in adata.obs['sample']]

In [ ]:
def preprocess_andata10x(adata_og,pct_mito=0.25,min_genes=200,max_genes=2400,min_counts=100,max_counts=4500):


    coding_list, _ = pull_gene_annots()


    # adata_og.var['mt'] = adata_og.var_names.str.startswith('mt-')
    # adata_og.var['coding'] = [gene in coding_list for gene in adata_og.var_names]
    # sc.pp.calculate_qc_metrics(adata_og, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

    adata_filt = adata_og.copy()

    adata_filt.var['mt'] = adata_filt.var_names.str.startswith('mt-')
    adata_filt.var['coding'] = [gene in coding_list for gene in adata_filt.var_names]
    sc.pp.calculate_qc_metrics(adata_filt, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    display(adata_filt.var.head())
    # adata_filt.var['not_mt'] =  np.logical_not( adata_filt.var['mt'])

    # logging.info('Filtering...')
    # adata_filt = adata_filt[adata_filt.obs.pct_counts_mt < pct_mito, :]
    # logging.info(str(np.sum(adata_og.obs.pct_counts_mt <pct_mito)) + f' cells with >{pct_mito}% mt-RNA removed')
    # adata_filt = adata_filt[:, adata_filt.var.coding]
    # logging.info(str(np.sum(np.logical_not(adata_og.var.coding))) + ' non coding genes removed')
    # sc.pp.filter_cells(adata_filt, min_genes=min_genes)
    # sc.pp.filter_cells(adata_filt, max_genes=max_genes)
    # sc.pp.filter_cells(adata_filt, min_counts=min_counts)
    # sc.pp.filter_cells(adata_filt, max_counts=max_counts)
    # fig,ax=plt.subplots(1,figsize=(1.5,1.5))
    # sc.pl.scatter(adata_filt, x='total_counts', y='n_genes_by_counts',ax=ax)
    # logging.info(str(len(adata_filt.obs))+' cells remaining')
    return adata_filt,adata_og

adata_filt = preprocess_andata10x(adata)[0]

In [ ]:

def high_var_genes_dim_reduc(adata,min_mean = 0.25,max_mean = 4,min_disp=0.55,pca_list=[],to_plot_pc=False):
    ''' The gene expression matrices were then log-normalized with a scale factor of 10,000,
    using the Seurat NormalizeData function57,58. Highly dispersed genes were selected using
    the Seurat FindVariableGenes function57,58,filtering for an average expression range of
    0.25–4 and a minimum dispersion of 0.55, resulting in a list of 2,197 genes.'''
    adata.raw = adata
    sc.pp.normalize_total(adata, target_sum=10000)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, min_mean=min_mean, max_mean=max_mean, min_disp=min_disp)
    with rc_context({'figure.figsize': (1.5, 1.5)}):
        sc.pl.highly_variable_genes(adata)
    plt.tight_layout()
    logging.info(str(np.sum(adata.var['highly_variable'])) + ' highly variable genes')


    #### PCA
    sc.tl.pca(adata, svd_solver='arpack',n_comps=50)
    fig,ax=plt.subplots(figsize=(1.5,1))
    ax.plot(adata.uns['pca']['variance_ratio'][:35],'ok',markersize=1)
    if to_plot_pc:
        quiet_PCA_plots(adata,pca_list,pc_pairs=[(0,1),(2,3),(4,5),(6,7)])

    return adata

def quiet_PCA_plots(adata,key_list,figsize=(1.5, 1.5),pc_pairs=[(0,1)]):
    fig,ax=plt.subplots(1*len(pc_pairs),len(key_list),figsize=(figsize[0]*len(key_list),figsize[1]*len(pc_pairs)))
    for ip,pair in enumerate(pc_pairs):
        if len(key_list) == 1: ax=[ax]
        for key_ind,key in enumerate(key_list):
            key_types = sorted(list(set( adata.obs[key] )))
            for k in key_types:
                is_k = adata.obs[key]==k
                ax[ip,key_ind].scatter(adata.obsm['X_pca'][is_k,pair[0]],adata.obsm['X_pca'][is_k,pair[1]],s=2,marker='.',linewidth=0,edgecolors=None,label=k)
                ax[ip,key_ind].set_xlabel(f'PC{pair[0]}')
                ax[ip,key_ind].set_ylabel(f'PC{pair[1]}')
            if len(key_types)<8: ax[ip,key_ind].legend(key_types,loc='best',markerscale=3)
            ax[ip,key_ind].set_title(key)
            plt.tight_layout()
    return None


adata_QC = high_var_genes_dim_reduc(adata_filt,min_mean = 0.05,max_mean = 6,min_disp=0.55,pca_list=[])

In [ ]:
############## GABAERGIC Filter #####################
adata_GABA = adata_QC.copy()
adata_GABA.obs['Gad1_pos'] = z_score(sc.get.obs_df(adata_GABA,'Gad1'))>0.5
adata_GABA.obs['Gad2_pos'] = z_score(sc.get.obs_df(adata_GABA,'Gad2'))>0.5
adata_GABA.obs['Syn1_pos'] = z_score(sc.get.obs_df(adata_GABA,'Syn1'))>-100
is_gaba = np.logical_and( np.logical_or(adata_GABA.obs['Gad1_pos'] , adata_GABA.obs['Gad2_pos'] ), adata_GABA.obs['Syn1_pos'])
adata_GABA.obs['Gabaergic'] = is_gaba
adata_GABA = adata_GABA[ adata_GABA.obs['Gabaergic'] ,:]
percent_gaba = np.sum(is_gaba)/len(is_gaba)*100
logging.info(str(len(adata_GABA.obs)) + f' Gabaergic cells {percent_gaba}%')

In [ ]:
def umap_and_cluster(adata, n_neighbors=15, n_pcs=25,resolution=.6,plot_keys=['Cluster (nn)'],size = 1,to_plot=True):
    sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs,random_state=42)
    sc.tl.louvain(adata,resolution=resolution,random_state=42)
    sc.tl.paga(adata)
    sc.tl.umap(adata,random_state=42)
    adata.obs['Cluster (nn)']= adata.obs['louvain']
    if to_plot:
        with rc_context({'figure.figsize': (2.5, 2.5)}):
            sc.pl.umap(adata,add_outline=False, legend_loc='on data', color=plot_keys,size=size)
    return adata

def explore_umap(adata_GABA,key_list=[],size=1,legend_loc=None):
    with rc_context({'figure.figsize': (1.5,1.5)}):
        sc.pl.umap(adata_GABA, legend_loc=legend_loc, color=key_list,vmin=0,size=size) # add_outline=True,
        plt.tight_layout()

def get_gene_cluster(adata,gene):
    gene_bool = z_score(sc.get.obs_df(adata,gene))>2
    gene_cluster_list = np.array(adata.obs['louvain'][gene_bool])
    gene_clust = st.mode(gene_cluster_list)
    return gene_clust[0][0]


adata_GABA = umap_and_cluster(adata_GABA, n_neighbors=15, n_pcs=15,resolution=.6,to_plot=False)
explore_umap(adata_GABA,['Cluster (nn)','Sst','Pvalb'],legend_loc='on data')
explore_umap(adata_GABA,['sample','treatment','apoe_iso'])
# explore_umap(adata_GABA,['Sst'])
adata_GABA.var

In [ ]:
def get_gene_cluster(adata,gene):
    gene_bool = z_score(sc.get.obs_df(adata,gene))>2
    logging.info(str(np.sum(gene_bool)) + ' ' + gene +' positive')
    gene_cluster_list = np.array(adata.obs['louvain'][gene_bool])
    gene_clust = st.mode(gene_cluster_list)
    return gene_clust[0][0]

sst_clust = get_gene_cluster(adata_GABA,'Sst')
pv_clust = get_gene_cluster(adata_GABA,'Pvalb')

print('sst_clust',sst_clust)
print('pv_clust',pv_clust)

In [ ]:
is_sst = z_score(sc.get.obs_df(adata_GABA,'Sst'))>0 ### use marker
is_sst = [sst_clust in l for l in sc.get.obs_df(adata_GABA,'louvain')] ### use cluster
is_bmx = ['BMX' in t for t in sc.get.obs_df(adata_GABA,'treatment')]
is_E4 = ['E4' in i for i in sc.get.obs_df(adata_GABA,'apoe_iso')]
# display(adata_GABA.obs.head(5))

BMX_E4 = np.logical_and(is_E4,is_bmx)
BMX_E3 = np.logical_and(np.logical_not(is_E4),is_bmx)
Veh_E4 = np.logical_and(is_E4,np.logical_not(is_bmx))
Veh_E3 = np.logical_and(np.logical_not(is_E4),np.logical_not(is_bmx))


BMX_E4 = np.sum(np.logical_and(BMX_E4,is_sst))
BMX_E3 = np.sum(np.logical_and(BMX_E3,is_sst))
Veh_E4 = np.sum(np.logical_and(Veh_E4,is_sst))
Veh_E3 = np.sum(np.logical_and(Veh_E3,is_sst))


data = [[Veh_E4, BMX_E4], [Veh_E3, BMX_E3] ]
# sst_fisher_df = pd.DataFrame(data=data+[[Veh_E4/Veh_E3, BMX_E4/BMX_E3]],index=['E4','E3','odds'],columns = ['VEH','BMX'])
sst_fisher_df = pd.DataFrame(data=data,index=['E4','E3'],columns = ['VEH','BMX'])
display(sst_fisher_df.head())
odd_ratio, p_value = stats.fisher_exact(data)
print(f'log odds ratio = {np.log(odd_ratio):.3}; p = {p_value:.3}')

In [ ]:
is_sst = z_score(sc.get.obs_df(adata_GABA,'Sst'))>1 ### use marker
is_sst = [sst_clust in l for l in sc.get.obs_df(adata_GABA,'louvain')] ### use cluster
is_bmx = ['BMX' in t for t in sc.get.obs_df(adata_GABA,'treatment')]
is_E4 = ['E4' in i for i in sc.get.obs_df(adata_GABA,'apoe_iso')]
# display(adata_GABA.obs.head(5))


import scipy.stats as stats


# creating data

BMX_SST = np.logical_and(is_sst,is_bmx)
BMX_other = np.logical_and(np.logical_not(is_sst),is_bmx)
Veh_SST = np.logical_and(is_sst,np.logical_not(is_bmx))
Veh_other = np.logical_and(np.logical_not(is_sst),np.logical_not(is_bmx))


BMX_SST = np.sum(np.logical_and(BMX_SST,is_E4))
BMX_other = np.sum(np.logical_and(BMX_other,is_E4))
Veh_SST = np.sum(np.logical_and(Veh_SST,is_E4))
Veh_other = np.sum(np.logical_and(Veh_other,is_E4))


data = [[Veh_SST, BMX_SST], [Veh_other, BMX_other]]
# E4_fisher_df = pd.DataFrame(data=data+ [[Veh_SST/Veh_other, BMX_SST/BMX_other]] ,index=['SST','Other','odds'],columns = ['VEH','BMX'])
E4_fisher_df = pd.DataFrame(data=data ,index=['SST','Other'],columns = ['VEH','BMX'])
display(E4_fisher_df.head())
odd_ratio, p_value = stats.fisher_exact(data)
print(f'log odds ratio = {np.log(odd_ratio):.3}; p = {p_value:.3}')



In [ ]:
### BMX SST DEG

def deg_set_up(adata,keys_of_interest):
    adata.raw = adata
    group_key = '_'.join(keys_of_interest)
    adata.obs[group_key] = adata.obs[keys_of_interest[0]]
    for i,v in enumerate(keys_of_interest[1:]):
        adata.obs[group_key] = adata.obs[group_key].str.cat(adata.obs[v].astype(str),sep='_')
    logging.info(list(set(adata.obs[group_key])))
    return adata,group_key


def deg_analysis(adata_temp,new_file_path,analysis_pairs,group_key,adata_name='volcano',log2fc_extrema=15,label_cutoff=1,to_plot=True):
    '''
    Takes in anndata RNA object and a stratifying key and returns
    DEGs based on given grouppings. Also builds volcano plots.    '''
    adata_temp.raw = adata_temp
    '''Perform statistics and write to DF'''
    try: os.makedirs(new_file_path)
    except: None
    # print(os.getcwd())
    deg_df_dict={}
    for key,pair in analysis_pairs.items():
        sc.tl.rank_genes_groups(adata_temp,use_raw=True,groupby=group_key,groups=pair,reference=pair[1],key_added=key,method='wilcoxon',tie_correct=True,n_genes=None)
        deg_df = sc.get.rank_genes_groups_df(adata_temp, group=pair[0],key=key,pval_cutoff=10,log2fc_min=-abs(log2fc_extrema), log2fc_max=abs(log2fc_extrema))
        deg_df = deg_df.set_index('names')
        deg_df_dict[key] = deg_df
        new_file_name = os.path.join(new_file_path,adata_name+'_'+key+'.csv')
        deg_df.to_csv(new_file_name)
    return adata_temp, deg_df_dict

### Just SST
adata_sst_cluster = adata_GABA.copy()[adata_GABA.obs['louvain']==sst_clust,:]
adata_sst_cluster,group_key = deg_set_up( adata_sst_cluster,['treatment','louvain'])
adata_temp, deg_df_dict=deg_analysis(adata_sst_cluster,'./BMX_DEG/',{'BMX_wrt_VEH':['BMX_1', 'VEH_1']},group_key,log2fc_extrema=20)

In [ ]:
def deg_volc_plot(deg_df_dict,gene_list=[],label_cutoff=0.1,suptitle='',lock_y_max=True,special_genes=[],use_adj=True):
    ''' Plot Volcanoes '''
    fig_volcano,ax=plt.subplots(1,len(deg_df_dict),figsize=(len(deg_df_dict)*1.5,2))
    if len(deg_df_dict) == 1:  ax=[ax]
    for key,df in deg_df_dict.items():
        key_ind = list(deg_df_dict.keys()).index(key)
        ax[key_ind].set_title(key)
        genes= df.index
        lfc = df['logfoldchanges']
        if use_adj: pval = df['pvals_adj']
        else: pval = df['pvals']

        neg_log10_pval = -np.log10(pval)
        for i,g in enumerate(genes):
            if str(g) in gene_list:
                if not np.isnan(lfc[i]*neg_log10_pval[i]):
                    ax[key_ind].scatter(lfc[i],neg_log10_pval[i],c='k',s=3)
                    if pval[i]<label_cutoff:
                        ax[key_ind].text(lfc[i],neg_log10_pval[i],genes[i],rotation=45,fontsize=4)

                if g in special_genes:
                    if not np.isnan(lfc[i]*neg_log10_pval[i]):
                        ax[key_ind].scatter(lfc[i],neg_log10_pval[i],c='r',s=3)
                        ax[key_ind].text(lfc[i],neg_log10_pval[i],genes[i],rotation=45,fontsize=4,color='r')


    x_lim=[]
    y_lim=[]
    for a in ax:
        x_lim.append(a.get_xlim())
        y_lim.append(a.get_ylim())

    x_etr = np.max(np.abs(x_lim))
    y_etr = np.max(np.abs(y_lim))

    for a in ax:
        a.set_xlim(-x_etr*1.1,x_etr*1.1)
        if lock_y_max: a.set_ylim(0,y_etr*1.1)
        else: a.set_ylim(0,a.get_ylim()[1]*1.1)
        a.set_xlabel('Log2 Fold Change')
        if use_adj: a.set_ylabel('-log10(adj-pvalue)')
        else: a.set_ylabel('-log10(raw-pvalue)')
        a.axhline(-np.log10(0.05),c='k',linestyle=":",linewidth=.6,label='p=0.05')

    fig_volcano.suptitle(suptitle)
    plt.tight_layout()
    return fig_volcano,ax

kcn_list = sorted([n for n in adata_GABA.var.index if 'kcn' in str(n).lower()])
fig_volcano,ax = deg_volc_plot(deg_df_dict,gene_list=kcn_list,suptitle='',lock_y_max=False,special_genes=['Kcnt2'],use_adj=False)
fig_volcano.set_size_inches(3, 2)

In [ ]:
explore_umap(adata_GABA,['Lamp5','Sst','Reln'],legend_loc='on data')
explore_umap(adata_GABA,['sample','treatment','apoe_iso'])
explore_umap(adata_GABA,['sample','treatment','apoe_iso'])